Buisness Problem:

Create a product that builds a Competitor Research Report for a company based on its website. The report should summarize:

What the company does (Products/Services)

Who its customers are

Pricing / Plans (if available)

Careers / Jobs info (hiring or not)

Unique value proposition

Any partnerships or notable achievements

In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from scraper import fetch_website_contents, fetch_website_links
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4.1-nano'
openai = OpenAI()

API key looks good so far


In [8]:
competitor_link_system_prompt = """
You are provided with a list of links extracted from a company website.
Identify links relevent for a Competitor Research Report.

Useful links include pages like:
-About Page/Company Page
-Products/Solutions/Services
-Pricing/Plan
-Customers or Case Studies
-Careers/Jobs
-Partners
-Press/News

Do not include: terms,privacy,email links,login.

Respond with a JSON object in format:
{"links":[{"type": "...", "url": "https://..." }]}
"""


In [9]:
def get_links_user_prompt(url):
    links = fetch_website_links(url)
    prompt = f"Website: {url}\nlinks:\n" + "\n".join(links)
    return prompt

In [13]:
def select_relevent_links(url):
    print(f"Selecting relevent links for (url)")
    response = openai.chat.completions.create(
        model= MODEL,
        messages=[
            {"role":"system","content":competitor_link_system_prompt},
            {"role":"user","content":get_links_user_prompt(url)}
        ],
        response_format={"type":"json_object"}
    )
    result=json.loads(response.choices[0].message.content)
    print(f"Found {len(result['links'])} relevent links.")
    return result

In [18]:
def fetch_all_relevant_page_content(url):
    landing = fetch_website_contents(url)
    relevent = select_relevent_links(url)

    result = f"## Landing Page\n\n{landing}"
    for link in relevent['links']:
        result += f"\n\n### {link['type'].title()}\n"
        result += fetch_website_contents(link['url'])
    return result[:5000] 

In [26]:
competitor_report_system_prompt = """
You are an AI that creates a easily understandable Competitor Analysis Report for a company.
Use headings and bullet points in clean Markdown.

Include these sections when possible:

- Overview of the Company
- Products / Services
- Customers & Use Cases
- Pricing / Plans
- Careers / Hiring
- Unique Strengths / Value Proposition
- Risks or Weaknesses (if visible)

Keep it professional and factual.
"""

In [27]:
def generate_competitor_report(company_name, url):
    content = fetch_all_relevant_page_content(url)

    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": competitor_report_system_prompt},
            {"role": "user", "content": f"Company: {company_name}\n\nWebsite Content:\n{content}"}
        ],
    )
    report = response.choices[0].message.content
    display(Markdown(report))

In [28]:
generate_competitor_report("Notion", "https://www.notion.com/")

Selecting relevent links for (url)
Found 6 relevent links.


# Competitor Analysis Report: Notion

---

## Overview of the Company
- **Name:** Notion  
- **Website:** https://www.notion.so  
- **Industry:** Productivity software / Workspace collaboration / AI-powered tools  
- **Mission:** To streamline work by combining teams and AI agents in a single workspace, reducing busywork and enhancing productivity.  
- **User Base:** Over 100 million users worldwide  
- **Recognition:**  
  - #1 knowledge base (G2, 3 years running)  
  - #1 AI enterprise search (G2)  
  - #1 rated AI writing (G2)  

---

## Products / Services
- **Notion 3.0:** Latest version emphasizing AI-enhanced productivity and collaboration.
- **Notion Agent:**  
  - AI assistant that performs tasks on behalf of users.  
  - Automates busywork tasks — what previously took days can be done in minutes.  
  - Collaborates with team members like a built-in power user.  
  - Personalizes behavior based on individual user preferences.  
  - Searches across all data sources: pages, messages, files, and the web.  
  - Custom Agents feature coming soon to automate repetitive tasks.

- **Enterprise Search:** Unified search tool covering all company data and files — one search for everything.
- **AI Meeting Notes:** Automatically creates perfect meeting notes.
- **Flexible Workflows:** Tools to manage projects of any size, emphasizing flexibility and scalability.
- **Unified Workspace:** Combines teams and tools under one platform to increase efficiency and reduce the number of apps needed.

---

## Customers & Use Cases
- **Target Users:** Teams and enterprises looking to centralize knowledge, automate workflows, and leverage AI to increase productivity.
- **Use Cases:**  
  - Knowledge management platform  
  - AI-powered project management and task automation  
  - Enterprise-wide search and data retrieval  
  - Collaborative document editing and note-taking  
  - Meeting transcription and note automation  
- **Customer Testimonials:**  
  - A 7-person team can function like 70 with Notion Agent.  
  - Clients report 3x faster workflows and improved team collaboration.  
  - Notion cited as a competitive AI-native tool giving advantages in productivity.

---

## Pricing / Plans
- The website prompts visitors to "See pricing plans," indicating tiered offerings but specific prices are not visible in the content.
- Emphasis on a free access tier ("Get Notion free") and the option to request a demo suggests freemium and enterprise pricing models.
- Value calculator tools are available to demonstrate savings from consolidating tools into Notion.

---

## Careers / Hiring
- No direct content about careers or hiring was included in the provided text.
- Given company scale (100M+ users) and continuous product development (Notion 3.0, AI Agents), likely ongoing recruitment for product, AI, engineering, and support roles.

---

## Unique Strengths / Value Proposition
- **All-in-One Workspace:** Combines knowledge management, AI automation, project management, and enterprise search in one platform.  
- **AI Integration:** Advanced AI agents that understand user behavior and automate tasks effectively.  
- **Personalization:** Customizable AI assistants that adapt to individual workflows.  
- **Trusted by Millions:** Large user base and high ratings on influential platforms (G2).
- **Enterprise Focus:** Features like enterprise search and automation appeal to large organizations seeking productivity gains.  
- **Proven Efficiency Gains:** Case studies and testimonials report significant productivity improvements and streamlined workflows.

---

## Risks or Weaknesses
- **Pricing Transparency:** Pricing models and plans are not immediately clear from available content, which may pose a barrier for some prospects.  
- **Heavy Dependence on AI:** While AI agents offer unique advantages, they may also create reliance risks if functionality is imperfect or training data is biased.  
- **Competition:** The productivity space is crowded with well-established tools (e.g., Microsoft Teams, Asana, Slack, Google Workspace) that also aggressively add AI capabilities.  
- **Adoption Curve:** Transitioning teams from their current toolchains can require significant change management.

---

# Summary
Notion positions itself as a transformative AI-first workspace solution designed to reduce busywork through intelligent automation and unified knowledge management. Its strengths lie in deep AI integration, a large user base, and an all-in-one platform approach targeting both small teams and enterprises. However, the company operates in a highly competitive sector where clear pricing transparency and continued innovation are vital for sustained growth.

---

*Report compiled based on publicly available website content as of June 2024.*